In [2]:
# Importowanie Bibliotek
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from kerastuner.tuners import BayesianOptimization

C:\Users\Micha\AppData\Local\Temp\ipykernel_33604\1122816733.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


In [3]:
# Przygotowanie Danych

# Wczytanie danych z pliku CSV
dane = pd.read_csv('../data.csv')

# Wybór tylko numerycznych kolumn do treningu
kolumny_numeryczne = ['Size(L)', 'OG', 'FG', 'IBU', 'ABV', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = dane[kolumny_numeryczne].copy()

# Uzupełnienie brakujących wartości średnią
X = X.fillna(X.mean())

# Przygotowanie etykiet (kolor piwa)
y = dane['Color'].copy()

# Normalizacja danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:
# Utworzenie Modelu
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=hp.Int('units_3', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])
    
    return model


In [5]:
tuner = BayesianOptimization(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=10,
    directory='../models',
    project_name='beer_color_prediction')

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='../models/beer_color_prediction')

tuner.search(X_train, y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback],
             batch_size=32)

Trial 10 Complete [00h 00m 41s]
val_mean_absolute_error: 7.514266490936279

Best val_mean_absolute_error So Far: 7.324926853179932
Total elapsed time: 00h 08m 28s


In [6]:
# Pobieranie najlepszego modelu
best_model = tuner.get_best_models(num_models=1)[0]

# Wyświetlanie podsumowania modelu
best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │       196,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 272,641 (1.04 MB)

 Trainable params: 270,337 (1.03 MB)

 Non-trainable params: 2,304 (9.00 KB)

In [7]:
# Trening najlepszego modelu
history = best_model.fit(X_train, y_train, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

# Wyświetlanie podsumowania modelu
best_model.summary()

final_val_mae = history.history['val_mean_absolute_error'][-1]
print(f"Final Validation MAE: {final_val_mae:.2f}")

Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 115.1386 - mean_absolute_error: 7.9381 - val_loss: 110.0328 - val_mean_absolute_error: 7.7999
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 116.3627 - mean_absolute_error: 7.9779 - val_loss: 113.0461 - val_mean_absolute_error: 8.2384
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 113.2944 - mean_absolute_error: 7.8702 - val_loss: 109.1389 - val_mean_absolute_error: 7.8198
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 112.8242 - mean_absolute_error: 7.8723 - val_loss: 108.3852 - val_mean_absolute_error: 7.5873
Epoch 5/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 114.8922 - mean_absolute_error: 7.8802 - val_loss: 109.2554 - val_mean_absolute_error: 7.6096
Epoch 6/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 113.4747 - mean_absolute_error: 7.8843 - val_loss: 109.6694 - val_mean_absolute_error: 7.5697
Epoch 7/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 11

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │       196,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,317 (3.10 MB)

 Trainable params: 270,337 (1.03 MB)

 Non-trainable params: 2,304 (9.00 KB)

 Optimizer params: 540,676 (2.06 MB)

Final Validation MAE: 7.97


In [1]:
%load_ext tensorboard
%tensorboard --logdir ../models/beer_color_prediction

Launching TensorBoard...